In [15]:
import torch
from ultralytics import YOLO
import pandas as pd

# Initialize your pose estimation model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
posemodel = YOLO("yolo11x-pose.pt")

In [16]:
class Video:
    # Class to load and process a video given its ID
    def __init__(self, video_id):
        self.video_id = video_id
        self.data = self.getdata()

    def getdata(self):
        print(f"Loading video {self.video_id} ...")
        try:
            # Process the video file from the DLdataset directory
            results = posemodel(f"DLdataset/{self.video_id}.mp4", iou=0.4, verbose=False, max_det=1, stream=True)
        except Exception as e:
            print(f"Error loading video {self.video_id}: {e}")
            return None

        tensors_list = []
        for result in results:
            try:
                # Extract normalized keypoints (shape: [17, 2])
                keypoints = result.keypoints.xyn.reshape(17, 2)
                tensors_list.append(keypoints)
            except Exception as e:
                print(f"Error processing frame in video {self.video_id}: {e}")
        if not tensors_list:
            print(f"No valid frames found in video {self.video_id}.")
            return None

        video_tensor = torch.stack(tensors_list)  # Shape: [num_frames, 17, 2]
        video_tensor = pad_sequence(video_tensor, max_length=200)  # Now shape: [200, 17, 2]
        print(f"Video {self.video_id} loaded.")
        return video_tensor

In [17]:
def pad_sequence(tensor, max_length=200):
    # Pad or truncate the input tensor along the first dimension to a fixed length
    num_frames = tensor.shape[0]
    if num_frames < max_length:
        padding = torch.zeros(max_length - num_frames, tensor.shape[1], tensor.shape[2], device=tensor.device)
        return torch.cat([tensor, padding], dim=0)
    return tensor[:max_length]

In [21]:
def horizontal_flip(stacked_tensor):
    # Returns a new tensor where x-coordinates values are flipped (1 - x) for nonzero values
    flipped = stacked_tensor.clone() # Clone the input tensor
    x = flipped[..., 0]  # Extract the x-coordinate values Shape: (N, 200, 17)
    mask = x != 0 # Create a mask when x is nonzero
    x[mask] = 1 - x[mask] # Apply the flip where the mask is True
    flipped[..., 0] = x # Update the flipped tensor's x-coordinates
    return flipped

In [23]:
def process_and_stack_videos(video_ids, output_file="all_videos.pt"):
    # Processes each video in video_ids using the Video class
    video_tensors = []
    for vid in video_ids:
        video_instance = Video(vid)
        data = video_instance.data
        if data is not None:
            video_tensors.append(data)
        else:
            print(f"Skipping video {vid} because it could not be processed.")
    
    if not video_tensors:
        raise RuntimeError("No videos were successfully processed.")
    
    # Stack the list of tensors into a single tensor
    stacked_tensor = torch.stack(video_tensors)  # Shape: (N, 200, 17, 2)
    flipped_tensor = horizontal_flip(stacked_tensor) # Shape: (N, 200, 17, 2)
    output_tensor = torch.cat((stacked_tensor,flipped_tensor), dim=0) # Shape: (N*2, 200, 17, 2)
    torch.save(output_tensor, output_file)
    print(f"Saved tensor with shape {output_tensor.shape} to {output_file}")

In [12]:
labels_csv_path = "DLdataset/simple_labels.csv"
video_ids = pd.read_csv(labels_csv_path)["ID"].tolist()
process_and_stack_videos(video_ids)

Loading video 1 ...
Video 1 loaded.
Loading video 2 ...
Video 2 loaded.
Loading video 3 ...
Video 3 loaded.
Loading video 4 ...
Video 4 loaded.
Loading video 5 ...
Video 5 loaded.
Loading video 6 ...
Video 6 loaded.
Loading video 7 ...
Video 7 loaded.
Loading video 8 ...
Video 8 loaded.
Loading video 9 ...
Video 9 loaded.
Loading video 10 ...
Video 10 loaded.
Loading video 11 ...
Video 11 loaded.
Loading video 12 ...
Video 12 loaded.
Loading video 13 ...
Video 13 loaded.
Loading video 14 ...
Video 14 loaded.
Loading video 15 ...
Video 15 loaded.
Loading video 16 ...
Video 16 loaded.
Loading video 17 ...
Video 17 loaded.
Loading video 18 ...
Video 18 loaded.
Loading video 19 ...
Video 19 loaded.
Loading video 20 ...
Video 20 loaded.
Loading video 21 ...
Video 21 loaded.
Loading video 22 ...
Video 22 loaded.
Loading video 23 ...
Video 23 loaded.
Loading video 24 ...
Video 24 loaded.
Loading video 25 ...
Video 25 loaded.
Loading video 26 ...
Video 26 loaded.
Loading video 27 ...
Video 27 